In [1]:
import torch
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Set prompt template for generation (optional)
from llama_index.core import PromptTemplate
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:

def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt

In [3]:
# Set Qwen1.5 as the language model and set generation config
model_name_or_path = "./model_local/qwen/Qwen1.5-7B-Chat"
Settings.llm = HuggingFaceLLM(
    model_name=model_name_or_path,
    tokenizer_name=model_name_or_path,
    context_window=30000,
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.38s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
embed_model_path = "./embedding_model"
# Set embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name = embed_model_path
)

# Set the size of the text chunk for retrieval
Settings.transformations = [SentenceSplitter(chunk_size=1024)]

本地文件夹中的文件（无论是PDF格式还是TXT格式）构建索引

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./test_data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    transformations=Settings.transformations
)

In [7]:
query_engine = index.as_query_engine()
your_query = "什么是安全管理关系模型？"
print(query_engine.query(your_query).response)

安全管理关系模型是一种组织和管理用户、用户组、角色、操作集以及资源分组之间权限关系的概念。在这个模型中：

1. **用户组**：代表一组共享相同安全权限的用户集合。
2. **用户**：是终端用户或者是通过API进行交互的实体，每个用户至少属于一个用户组，如果没有特别指定，默认属于系统默认用户组。
3. **角色**：定义了系统中的权限范围，包括管理的资源范围和可执行的操作范围。角色被分配给用户后，用户便拥有了这些角色所涵盖的权限。
4. **操作集**：是系统中具体操作（如增删改查）的权限集合，是角色权限的一部分。
5. **资源**：与业务相关联，比如图标、菜单等，可以作为权限管理的对象。

这个模型通过这样的层级结构来确保对系统资源的访问和操作得到有效的控制，以保护数据和系统安全。


以下代码片段展示了如何为一系列网站的内容构建索引。

In [10]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://qwen.readthedocs.io/zh-cn/latest/framework/LlamaIndex.html"]
)
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    transformations=Settings.transformations
)

ConnectionError: HTTPSConnectionPool(host='qwen.readthedocs.io', port=443): Max retries exceeded with url: /zh-cn/latest/framework/LlamaIndex.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f07ac1661d0>: Failed to resolve 'qwen.readthedocs.io' ([Errno -3] Temporary failure in name resolution)"))

### 检索增强（RAG）
现在您可以输入查询，Qwen1.5 将基于索引文档的内容提供答案。

In [11]:
query_engine = index.as_query_engine()
your_query = "如何保存和加载已经构建的检索"
print(query_engine.query(your_query).response)

在没有具体上下文的情况下，根据提供的信息，似乎是在谈论某种类型的检索系统或应用程序的构建过程。要保存和加载已经构建的检索，通常会使用数据持久化技术和编程语言中的序列化机制。以下是一个通用的步骤概述：

1. **保存检索结果**：
   - **数据库存储**：如果检索结果是基于数据库的操作，例如搜索结果、用户设置等，你可以将它们保存到关系型数据库（如MySQL、PostgreSQL）或NoSQL数据库（如MongoDB）中。
   - **文件系统**：对于结构化的数据，如JSON、XML或二进制格式，可以保存为文件，如`.json`、`.xml` 或 `.bin`。
   - **缓存**：在内存中，可以使用缓存技术如Redis或Memcached来临时存储最近使用的检索结果，提高性能。

2. **加载检索**：
   - **数据库恢复**：从数据库中读取保存的数据，使用SQL查询或相应的API。
   - **文件解析**：如果文件格式是可读的，例如JSON、XML，直接解析文件内容。
   - **缓存回填**：如果检索结果已保存在缓存中，优先从缓存中加载，然后更新数据库以保持最新状态。

3. **代码实现**：
   - **编程语言**：使用适当的语言（如Python、Java、JavaScript等）编写函数来保存和加载检索数据，这些函数通常会涉及到文件操作、数据库操作或使用特定的库或框架。

4. **版本控制**：如果你需要在多个版本之间切换检索，可能还需要考虑版本控制，如Git，来管理不同版本的构建。

请注意，具体的实现方式会依赖于你使用的系统、平台和技术栈。如果有更详细的上下文，如使用的编程语言或框架，可以提供更精确的指导。


In [13]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
QWEN_7B = "./model_local/qwen/Qwen1.5-7B-Chat"

In [14]:

selected_model = QWEN_7B
 
SYSTEM_PROMPT = """你是一个人工智能助手，能根据给定的源文件友好地回答问题。以下是你要始终遵循的一些规则：
- 生成人类可读的输出结果，避免创建包含胡言乱语文本的输出结果。
- 只生成所要求的输出，不要在所要求的输出前后添加任何其他语言。
- 不要说 "谢谢"、"很高兴为您提供帮助"、"您是人工智能代理 "等。直接回答即可。
- 生成通常用于北美商业文档的专业语言。
- 绝不生成冒犯性语言或粗话.
"""
 
query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)
 
llm = HuggingFaceLLM(context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.06s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 加载词向量

In [15]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [16]:
embedding_model = "./embedding_model"
embed_model = HuggingFaceEmbedding(model_name=embedding_model)

In [17]:
from llama_index.core import Settings
 
Settings.llm = llm
Settings.embed_model = embed_model

In [18]:
from llama_index.core import SimpleDirectoryReader
# from llama_index.core import 
documents = []
for roots, dirs, files in os.walk("./ops/zedx2txt"):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(roots, file)
            dir_name = os.path.dirname(file_path)
            document_i = SimpleDirectoryReader(dir_name).load_data()
            documents.append(document_i)

KeyboardInterrupt: 

: 

In [ ]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("")
print(response)